In [5]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

UNIT_FACTORS = {
    'Feet': 0.092903,
    'Meters': 1.0,
    'Inches': 0.00064516,
    'Centimeters': 0.0001
}

UNIT_ABBREVIATIONS = {
    'Feet': 'ft²',
    'Meters': 'm²',
    'Inches': 'in²', 
    'Centimeters': 'cm²'
}

CURRENCY_SYMBOLS = {
    'USD': '$',
    'EUR': '€',
    'GBP': '£',
    'CAD': 'C$',
    'AUD': 'A$',
    'INR': '₹',
    'JPY': '¥',
    'Other': ''
}

# ─── INPUT WIDGETS (Optimized widths) ───
unit_dropdown = widgets.Dropdown(
    options=['Feet', 'Meters', 'Inches', 'Centimeters'],
    value='Meters',
    description='Units:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='220px')
)

width_input = widgets.FloatText(value=10, description='Width:', style={'description_width': '70px'}, layout={'width': '220px'})
height_input = widgets.FloatText(value=12, description='Height:', style={'description_width': '70px'}, layout={'width': '220px'})

# ─── COSTS WITH SINGLE CURRENCY SELECTOR ───
tile_cost_input = widgets.FloatText(value=5.50, description='Tile Cost:', style={'description_width': '90px'}, layout={'width': '150px'})
tile_currency = widgets.Dropdown(options=list(CURRENCY_SYMBOLS.keys()), value='USD', layout={'width': '88px'}, description='per m²')

material_cost_input = widgets.FloatText(value=0, description='Materials:', style={'description_width': '90px'}, layout={'width': '150px'})

labor_cost_input = widgets.FloatText(value=0, description='Labor:', style={'description_width': '90px'}, layout={'width': '150px'})

currency_dropdown = widgets.Dropdown(
    options=list(CURRENCY_SYMBOLS.keys()),
    value='USD',
    description='Currency:',
    style={'description_width': '80px'},
    layout={'width': '220px'}
)

calculate_btn = widgets.Button(
    description='Calculate Total',
    button_style='success',
    icon='calculator',
    layout={'width': '220px', 'margin': '10px 0 0 0'}
)

# function to update "per [unit]²" field
def update_tile_unit_description(change):
    unit = change['new']
    tile_currency.description = f'per {UNIT_ABBREVIATIONS[unit]}'

# observer to update "per [unit]²" field
unit_dropdown.observe(update_tile_unit_description, names='value')

# ─── OUTPUT AREA (Reduced width) ───
output = widgets.Output(
    layout=widgets.Layout(
        border='1px solid #d0d0d0',
        padding='15px',
        margin='0 0 0 25px',  # Add space between panels
        width='350px'  # Narrower output panel
    )
)

# ─── LAYOUT ───
input_panel = widgets.VBox([
    widgets.Label(value='📐 Floor Dimensions'),
    unit_dropdown,
    width_input,
    height_input,
    widgets.HTML(value='<hr style="margin: 15px 0;">'),
    widgets.Label(value='💰 Enter Costs'),
    currency_dropdown,
    tile_cost_input,
    material_cost_input,
    labor_cost_input,
    calculate_btn
], layout=widgets.Layout(padding='10px', width='340px'))

# ─── INFO BOX ───
info_box = widgets.HTML(value="""
<div style="background-color: #e8f4f8; padding: 10px; border-left: 4px solid #3498db; font-size: 0.85em; margin-bottom: 15px;">
    <strong>💡 Important:</strong> Currency conversion is NOT automatic. Enter costs in the same currency for accurate totals.
</div>
""")

# ─── DISPLAY EVERYTHING ───
display(info_box, widgets.HBox([input_panel, output]))

# ─── CALCULATION LOGIC ───
def calculate(btn):
    with output:
        clear_output()
        try:
            unit = unit_dropdown.value
            display_cur = currency_dropdown.value
            symbol = CURRENCY_SYMBOLS[display_cur]
            cur = currency_dropdown.value
            
            # Get values
            width = width_input.value
            height = height_input.value
            tile_cost = tile_cost_input.value
            material_cost = material_cost_input.value
            labor_cost = labor_cost_input.value
            
            # Validation
            if width <= 0 or height <= 0:
                display(HTML("<p style='color: red;'>⚠️ Dimensions must be positive.</p>"))
                return
            if tile_cost < 0 or material_cost < 0 or labor_cost < 0:
                display(HTML("<p style='color: red;'>⚠️ Costs cannot be negative.</p>"))
                return
            
            # Calculate area
            area_original = width * height
            area_sqft = area_original * UNIT_FACTORS[unit]
            
            # Costs with wastage
            wastage = 1.10
            tile_total = area_sqft * tile_cost * wastage
            grand_total = tile_total + material_cost + labor_cost
            
            # Display results
            html = f"""
            <div style="font-family: 'Segoe UI', Arial; font-size: 0.95em;">
                <h3 style="color: #2c3e50; margin-top:0;">📊 Cost Breakdown</h3>
                <table style="width:100%; border-collapse: collapse;">
                    <tr><td style="padding:6px 0;">Area ({unit}²):</td><td style="text-align:right;"><strong>{area_original:.2f}</strong></td></tr>
                    <tr><td colspan="2"><hr style="margin:8px 0;" /></td></tr>
                    <tr><td style="padding:6px 0;">Tile Cost:</td><td style="text-align:right;">{CURRENCY_SYMBOLS[cur]}{tile_total:,.2f}</td></tr>
                    <tr><td style="padding:6px 0;">Materials:</td><td style="text-align:right;">{CURRENCY_SYMBOLS[cur]}{material_cost:,.2f}</td></tr>
                    <tr><td style="padding:6px 0;">Labor:</td><td style="text-align:right;">{CURRENCY_SYMBOLS[cur]}{labor_cost:,.2f}</td></tr>
                    <tr><td colspan="2"><hr style="border-top:2px solid #3498db; margin:8px 0;" /></td></tr>
                    <tr><td style="padding:8px 0; font-size:1.1em;"><strong>GRAND TOTAL:</strong></td>
                        <td style="text-align:right; font-size:1.2em; color:#27ae60;">
                            <strong>{symbol}{grand_total:,.2f}</strong>
                            <span style="color:#7f8c8d; font-size:0.7em; display:block;">(Display: {display_cur})</span>
                        </td>
                    </tr>
                </table>
            </div>
            """
            display(HTML(html))
            
        except Exception as e:
            display(HTML(f"<p style='color: red;'>❌ Error: {str(e)}</p>"))

calculate_btn.on_click(calculate)

HTML(value='\n<div style="background-color: #e8f4f8; padding: 10px; border-left: 4px solid #3498db; font-size:…